<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part6_pydantic_output_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom Search Agent - Twitter Profile**



[Pydantic parser](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pydantic/)

In [ ]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [ ]:
%%capture --no-stderr
%pip install ChatGoogleGenerativeAI

In [ ]:
# packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [ ]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key, temperature=0)

In [ ]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [ ]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4fbdfd54-0d96-4f74-8839-c40536c50b8b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
# # Restart Runtime and Re-authenticate
# from google.colab import auth
# auth.authenticate_user()


In [ ]:
result = llm.invoke("hello")
print(result)

content='Hello there! How can I help you today?\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-bd42b149-b96c-410a-a94e-9de062ad3b64-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")
# model.invoke("hi")

[Tavily Search](https://python.langchain.com/docs/integrations/tools/tavily_search/)

[TAVILY_API_KEY](https://tavily.com/)


In [ ]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

[hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)



**Create React Agent**

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults


def get_profile_url_tavily(name: str):
    """Searches for Linkedin or Twitter Profile Page."""
    search = TavilySearchResults()
    res = search.run(f"{name}")
    return res

In [ ]:
from dotenv import load_dotenv

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor,
)
from langchain import hub
# from tools.tools import get_profile_url_tavily


def lookup(name: str) -> str:
    # llm = ChatOpenAI(
    #     temperature=0,
    #     model_name="gpt-4o-mini",
    # )
    template = """
       given the name {name_of_person} I want you to find a link to their Twitter profile page, and extract from it their username
       In Your Final answer only the person's username"""
    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )
    tools_for_agent = [
        Tool(
            name="Crawl Google 4 Twitter profile page",
            func=get_profile_url_tavily,
            description="useful for when you need get the Twitter Page URL",
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)

    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(name_of_person=name)}
    )

    twitter_username = result["output"]
    return twitter_username


# if __name__ == "__main__":
#     print(lookup(name="Elon Musk"))

In [ ]:
# # Example function call
print("Twitter Lookup ")
twitter_lookup = lookup(name="Elon Musk")
print("Twitter Lookup:", twitter_lookup)

Twitter Lookup 


> Entering new AgentExecutor chain...
Thought: I need to find Elon Musk's Twitter profile URL to extract his username.  I can use the "Crawl Google 4 Twitter profile page" action for this.

Action: Crawl Google 4 Twitter profile page
Action Input: Elon Musk
[{'url': 'https://www.amazon.com/Elon-Musk-SpaceX-Fantastic-Future/dp/0062301233', 'content': "In the spirit of Steve Jobs and Moneyball, Elon Musk is both an illuminating and authorized look at the extraordinary life of one of Silicon Valley's most exciting, unpredictable, and ambitious entrepreneurs--a real-life Tony Stark--and a fascinating exploration of the renewal of American invention and its new ``makers.''"}, {'url': 'https://www.tesla.com/elon-musk', 'content': "As the co-founder and CEO of Tesla, Elon leads all product design, engineering and global manufacturing of the company's electric vehicles, battery products and"}, {'url': 'https://www.forbes.com/profile/elon-musk/', 'content': "And Musk Of Profit

In [ ]:
# from dotenv import load_dotenv
from langchain.prompts.prompt import PromptTemplate


# from third_parties.linkedin import scrape_linkedin_profile
# from agents.linkedin_lookup_agent import lookup as linkedin_lookup_agent

# from dotenv import load_dotenv
# from langchain_google_genai import ChatGoogleGenerativeAI
# from pydantic import  SecretStr
# import os

# load_dotenv()

# gemini_api_key = SecretStr(os.environ["GEMINI_API_KEY"])

# if not gemini_api_key:
#     raise ValueError("GEMINI_API_KEY is not set. Please provide the API key in your environment.")

# print("API Key:", gemini_api_key)

# # Initialize the ChatGoogleGenerativeAI model
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",  # Specify the model
#     api_key=gemini_api_key,  # Pass the API key
#     )


def ice_break_with(name: str) -> str:

    # linkedin_username = linkedin_lookup_agent(name=name)
    # linkedin_data = scrape_linkedin_profile(linkedin_profile_url=linkedin_username)

    twitter_username = lookup(name=name)
    # tweets = scrape_user_tweets(username=twitter_username)

    summary_template = """
    given the Linkedin information {information}, and their lates twiter {twitter_post} I want you to create:
    1. A short summary
    2. two interesting facts about them

    Use both information from twitter and Linkedin
    """

    summary_prompt_template = PromptTemplate(
        input_variables=["information", "twitter_posts"], template=summary_template
    )

    # llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

    chain = summary_prompt_template | llm

    # res = chain.invoke(input={"information": linkedin_data, "twitter_posts": tweets})
    res = chain.invoke(input={"twitter_posts": twitter_username})

    print(res)


In [ ]:
# # Example function call
print("Twitter tweet ")
twitter_tweets = lookup(name="Harrison Chase")
print("Twitter Lookup:", twitter_tweets)

Twitter tweet 


> Entering new AgentExecutor chain...
Thought: I need to find Harrison Chase's Twitter profile URL to extract their username.  I can use the "Crawl Google 4 Twitter profile page" tool for this.

Action: Crawl Google 4 Twitter profile page
Action Input: Harrison Chase
[{'url': 'https://www.bigdatawire.com/people-to-watch-2024-harrison-chase/', 'content': 'People to Watch 2024 - Harrison Chase - BigDATAwire People to Watch 2024 – Harrison Chase Harrison Chase is the co-founder and CEO of LangChain, developer of the eponymous tool for chaining together interoperable components for GenAI applications. This website uses cookies to improve your experience. Privacy & Cookies Policy Out of these, the cookies that are categorized as necessary are stored on your browser as they are essential for the working of basic functionalities of the website. Necessary cookies are absolutely essential for the website to function properly. Any cookies that may not be particularly necessary f